[Reference](https://betterprogramming.pub/build-an-inventory-management-python-cli-application-d656b3ff8639)

In [2]:
import sqlite3
from model import Inventory

db_connection = sqlite3.connect('inventories.db')
cursor = db_connection.cursor()

def create_table():
    cursor.execute("""CREATE TABLE IF NOT EXISTS inventories(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name    text,
        category    text,
        quantity    integer,
        created_at  text,
        updated_at  text
        )""")

create_table()

def get_all_stocks():
    cursor.execute('select * from inventories')
    all_stocks = cursor.fetchall()
    stocks = []
    for stock in all_stocks:
        stocks.append(Inventory(stock[1], stock[2], stock[3], stock[4], stock[5], stock[0]))
    return stocks

def insert_stock(stock: Inventory):
    cursor.execute("SELECT * FROM inventories WHERE LOWER(name) = ?", (stock.name.lower(),))
    results = cursor.fetchall()
    if len(results):
        existing_stock = results[0]
        print(existing_stock)
        new_quantity = existing_stock[3] + stock.quantity
        id = existing_stock[0]
        with db_connection:
            cursor.execute("UPDATE inventories SET quantity=:quantity WHERE id=:id", {'id': id, 'quantity': new_quantity})
    else:
        with db_connection:
            cursor.execute(
                'INSERT INTO inventories(name, category, created_at, updated_at, quantity) VALUES(?,?,?,?,?)',
                (stock.name, stock.category, stock.created_at, stock.updated_at, stock.quantity)
            )

def reduce_stock_quantity(id: int, quantity: str):
    cursor.execute("SELECT * FROM inventories WHERE id=:id", {'id': id})
    stock = cursor.fetchone()
    if not stock:
        print("Stock not valid")
        return
    else:
        new_stock_quantity = stock[3] - quantity
        if new_stock_quantity < 0:
            print('Insufficient stock')
            return
        else:
            with db_connection:
                cursor.execute("UPDATE inventories SET quantity=:quantity WHERE id=:id", {'id': id, 'quantity': new_stock_quantity})

def increase_stock_quantity(id: int, quantity: str):
    cursor.execute("SELECT * FROM inventories WHERE id=:id", {'id': id})
    stock = cursor.fetchone()
    if not stock:
        print("Stock not valid")
        return
    else:
        new_stock_quantity = stock[3] + quantity
        with db_connection:
                cursor.execute("UPDATE inventories SET quantity=:quantity WHERE id=:id", {'id': id, 'quantity': new_stock_quantity})

def delete_stock(id):
    with db_connection:
        cursor.execute("DELETE from inventories WHERE id=:id", {'id': id})

In [1]:

import typer
from rich.console import Console
from rich.table import Table

from repository import get_all_stocks, insert_stock, reduce_stock_quantity, increase_stock_quantity, delete_stock
from model import Inventory

console = Console()
app = typer.Typer()

@app.command(short_help="show all stocks")
def add(name: str, quantity: int, category: str):
    typer.echo(f"Adding new stock...")
    new_stock = Inventory(name, category, quantity)
    insert_stock(new_stock)
    display()

@app.command(short_help="reduce stock quantity")
def reduce_stock(id: int, quantity: int):
    typer.echo(f"Reducing stock quantity...")
    reduce_stock_quantity(id, quantity)
    display()

@app.command(short_help="increase stock quantity")
def increase_stock(id: int, quantity: int):
    typer.echo(f"Increasing stock quantity...")
    increase_stock_quantity(id, quantity)
    display()

@app.command(short_help="clear stock")
def clear_stock(id: int):
    typer.echo(f"Clearing stock...")
    delete_stock(id)
    display()

@app.command(short_help="show all stocks")
def display():
    stocks = get_all_stocks()
    console.print("[bold magenta]-=-=-=-=-=-Inventory management application-=-=-=-=-=-[/bold magenta]",":notebook_with_decorative_cover:")

    table = Table(show_header=True, header_style="bold blue")
    table.add_column("id", style="dim", width=6)
    table.add_column("Name", min_width=20)
    table.add_column("Category", min_width=20, justify="center")
    table.add_column("Quantity", min_width=12, justify="center")
    table.add_column("Created date", min_width=20, justify="center")
    table.add_column("Status", min_width=20, justify="center")


    for _, stock in enumerate(stocks):
        depleted = ":red_circle:" if int(stock.quantity) < 1 else ":green_circle:"
        table.add_row(str(stock.id), stock.name, stock.category, str(stock.quantity), stock.created_at, depleted)
    console.print(table)


if __name__ == "__main__":
    app()